# Stellantis raw time series EDA
The goals of this notebook are to:
- visualize and get an intuition of how the stellantis data is like.  
- Find a lead to estimate the soh.

## Imports

In [ ]:
import plotly.express as px
from scipy import stats
import numpy as np
from core.stats_utils import *
from core.pandas_utils import *
from core.config import valid_soh_points
# from core.stats_utils import filter_results
from transform.fleet_info.main import fleet_info
from transform.raw_tss.main import get_raw_tss

## Setup

In [ ]:
tss = get_raw_tss('stellantis', force_update=False)


## Time series processing

In [ ]:
tss.columns

In [ ]:
tss.count() / len(tss)

In [ ]:
import plotly.graph_objects as go

def twinx(df, cols_y1, cols_y2, x_col=None):
    """
    Creates a Plotly figure with two y-axes (twin y-axis plot).
    
    Parameters:
    - df: pd.DataFrame - The dataframe containing the data.
    - cols_y1: list - List of column names for the primary y-axis (left side).
    - cols_y2: list - List of column names for the secondary y-axis (right side).
    - x_col: str (optional) - The column name to be used for the x-axis. If not provided, index is used.
    
    Returns:
    - fig: go.Figure - Plotly figure with dual y-axis.
    """
    fig = go.Figure()

    # Determine the x-axis data
    if x_col is None:
        x_data = df.index
    else:
        x_data = df[x_col]
    
    # Add traces for the first (left) y-axis
    for col in cols_y1:
        fig.add_trace(go.Scatter(x=x_data, y=df[col], name=col, yaxis="y1"))

    # Add traces for the second (right) y-axis
    for col in cols_y2:
        fig.add_trace(go.Scatter(x=x_data, y=df[col], name=col, yaxis="y2"))

    # Update layout for dual y-axis
    fig.update_layout(
        yaxis=dict(title="Primary Y-Axis", titlefont=dict(color="blue")),
        yaxis2=dict(
            title="Secondary Y-Axis",
            titlefont=dict(color="red"),
            overlaying="y",
            side="right"
        ),
        xaxis=dict(title=x_col if x_col else "Index")
    )

    return fig


In [ ]:
ts = tss[tss['vin'] == "VR1UJZKXZLW018492"]
COLS_TO_DISPLAY = [
        'vin', 
        'datetime', 
        'odometer', 
]
# twinx(ts, ['odometer'], x_col="datetime")
px.scatter(ts, x="datetime", y="odometer").show()

In [ ]:
ts["yes"] = ts["charging.estimated_range"] / (ts["charging.max_range"] * ts["charging.battery_level"])
px.scatter(ts, x="charging.battery_level", y=["charging.estimated_range", "charging.max_range"]).show()
px.scatter(ts, x="charging.battery_level", y=["yes"]).show()

In [ ]:
raw_tss["yes"] = raw_tss["charging.estimated_range"] / (raw_tss["charging.max_range"] * raw_tss["charging.battery_level"])
# px.scatter(raw_tss, x="charging.battery_level", y=["charging.estimated_range", "charging.max_range"]).show()
px.scatter(raw_tss, x="charging.battery_level", y=["yes"], color="vin").show()
px.scatter(raw_tss, x="diagnostics.odometer", y=["yes"], color="vin").show()